In [ ]:
import pandas as pd
import datetime
from datetime import timedelta
import os
import yaml
from pcse.models import Wofost73_WLP_MLWB
from pcse.input import YAMLCropDataProvider
from pcse.input import ExcelWeatherDataProvider
from pcse.base import ParameterProvider

### Load Input Data & Dictionaries

In [ ]:
### Weather Data
weatherfile = "Thesis Data/REH_WeatherData_PCSEFormat.xlsx"
wdp = ExcelWeatherDataProvider(weatherfile)
print(wdp)

In [ ]:
### Crop Data
cropd = YAMLCropDataProvider(Wofost73_WLP_MLWB)
cropd.set_active_crop('maize', 'Grain_maize_201')
print(cropd)

In [ ]:
### Site Data Dictionary
## Dictionary Function
def calculate_site_data_with_notinf(year, wav_file, fc_file, ssi_file, ssmax_file, notinf_file):
    """
    Calculates the site parameters WAV, SMLIM, SSI, and SSMAX for a given year,
    skipping years with no soil data (e.g., 2014).

    Parameters:
        year (int): The year for which to calculate the site parameters.
        wav_file (str): Path to the WAV results CSV file.
        fc_file (str): Path to the FC data file.
        ssi_file (str): Path to the SSI results CSV file.
        ssmax_file (str): Path to the SSMAX results CSV file.

    Returns:
        dict: Dictionary containing the calculated site parameters, or None if the year is skipped.
    """
    available_years = [2010, 2011, 2012, 2013, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
    if year not in available_years:
        print(f"No soil data available for year {year}. Skipping...")
        return None

    # WAV
    wav_data = pd.read_csv(wav_file)
    wav = wav_data.loc[wav_data['Year'] == year, 'WAV'].values[0]

    #SMLIM
    fc_data = pd.read_csv(fc_file)
    top_soil_row = (available_years.index(year)) * 3  # Adjust index for missing 2014
    smlim = fc_data.loc[top_soil_row, 'FC_PTF02']

    # SSI
    ssi_data = pd.read_csv(ssi_file)
    ssi = ssi_data.loc[ssi_data['Year'] == year, 'SSI (cm)'].values[0]

    # SSMAX
    ssmax_data = pd.read_csv(ssmax_file)
    ssmax = ssmax_data.loc[ssmax_data['Year'] == year, 'SSMAX (cm)'].values[0]

   # NOTINF
    notinf_data = pd.read_csv(notinf_file)
    notinf = notinf_data.loc[notinf_data['Year'] == year, 'NOTINF'].values[0]

    # Compile
    site_data = {
        "WAV": wav,
        "CO2": 400,  
        "IFUNRN": 1,  
        "NOTINF": notinf, 
        "SSI": ssi,
        "SMLIM": smlim,
        "SSMAX": ssmax
    }

    return site_data

# File paths
wav_file = "wav_results_2010_2023_RH.csv"
fc_file = "Thesis Data/pred_FC_RH.csv"
ssi_file = "ssi_results_2010_2023_RH.csv"
ssmax_file = "ssmax_results_2010_2023_RH.csv"
notinf_file = "notinf_results_2010_2023_RH.csv"

In [ ]:
### Agromanagement Dictionary
## Load the Sowing and Site Data
file_path = "Thesis Data/RH_VarTestingData.xlsx"
df = pd.read_excel(file_path)
df['sowing_date'] = pd.to_datetime(df['sowing_date'], format="%d.%m.%Y") # Ensure the 'sowing_date' column is in datetime format
df['harvest_date'] = pd.to_datetime(df['harvest_date'], format="%d.%m.%Y") # Ensure the 'harvest_date' column is in datetime format

## Dictionary Function
def get_agromanagement_maize(year, sowing_date, harvest_date, crop_name="maize", variety_name="middle"):
    """
    Generates a YAML-style agromanagement dictionary for maize.
    """
    sowing_date_parsed = pd.to_datetime(sowing_date).date() # Ensure sowing and harvest dates are Python date objects
    harvest_date_parsed = pd.to_datetime(harvest_date).date()
    campaign_start_date = datetime.date(year, 1, 1) # Define the campaign start date as the first day of the year

    # Generate YAML agromanagement structure dynamically
    agromanagement = {
        "AgroManagement": [
            {
                datetime.date(year, 1, 1): {  # Campaign start as a date object
                    "CropCalendar": {
                        "crop_name": crop_name,
                        "variety_name": variety_name,
                        "crop_start_date": sowing_date_parsed,  # Date object
                        "crop_start_type": "sowing",
                        "crop_end_date": harvest_date_parsed,    # Date object
                        "crop_end_type": "harvest",
                        "max_duration": 300
                    },
                    "TimedEvents": None,
                    "StateEvents": None
                }
            }
        ]
    }
    return agromanagement

In [ ]:
### Soil Data
def get_soil_data(soil_folder="soil_yamls_RH"):
    """
    Creates a dictionary mapping years to their corresponding soil YAML data.
    
    Parameters:
        soil_folder (str): Path to the folder containing soil YAML files (default: "soil_yamls").
    
    Returns:
        dict: A dictionary where the keys are years and the values are soil data.
    """
    soil_data_dict = {}
    
    # Iterate through all files in the soil folder
    for filename in os.listdir(soil_folder):
        if filename.endswith(".yaml"):
            year = int(filename.split("_")[1].split(".")[0]) # Extract the year from the filename
            file_path = os.path.join(soil_folder, filename) # Load the YAML file
            with open(file_path, "r") as file:
                soil_data = yaml.safe_load(file)
            soil_data_dict[year] = soil_data # Add the soil data to the dictionary
    
    return soil_data_dict

def get_soil_for_year(soil_data_dict, year):
    """
    Retrieves soil data for a specific year.
    
    Parameters:
        soil_data_dict (dict): Dictionary mapping years to soil data.
        year (int): The year to retrieve soil data for.
    
    Returns:
        dict: Soil data for the specified year.
    """
    if year not in soil_data_dict:
        raise ValueError(f"Soil data for year {year} not found.")
    return soil_data_dict[year]

In [ ]:
# Patch the WaterBalanceLayered class to include the NINFTB method
from pcse.soil.multilayer_waterbalance import WaterBalanceLayered

def ninftb_function(self, rain):
    if rain <= 0.5:
        return 0.0  # All rain infiltrates
    elif rain >= 1.5:
        return 1.0  # Maximum non-infiltrating fraction
    else:
        return (rain - 0.5) / (1.5 - 0.5)  # Linear interpolation

# Add the function as a method to the class
WaterBalanceLayered.NINFTB = ninftb_function


### Run Batch Mode

In [ ]:
# Output directory for results
output_dir = "wofost_outputs_early"
os.makedirs(output_dir, exist_ok=True)

# Load soil data dictionary
soil_data_dict = get_soil_data(soil_folder="soil_yamls_RH")

# Define the specific years to run
years = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

# Results container
results = []

# Define the variety name
valid_variety_name = 'Grain_maize_201'

for year in years:
    print(f"\nRunning WOFOST for year {year}...")

    # Filter for the specific year and series 'early'
    filtered_row = df[(df['year'] == year) & (df['series'] == 'early')]
    if filtered_row.empty:
        print(f"No data for year {year}. Skipping...")
        continue

    # Generate agromanagement dynamically
    sowing_date = filtered_row['sowing_date'].values[0]
    harvest_date = filtered_row['harvest_date'].values[0]
    agromanagement = get_agromanagement_maize(
        year, sowing_date, harvest_date, crop_name='maize', variety_name=valid_variety_name
    )

    # Generate soil data
    soil_data = get_soil_for_year(soil_data_dict, year)

    # Generate site data
    site_data = calculate_site_data_with_notinf(
        year, wav_file, fc_file, ssi_file, ssmax_file, notinf_file
    ) 
    if site_data is None:
        print(f"No site data for year {year}. Skipping...")
        continue

    # ParameterProvider
    params = ParameterProvider(cropdata=cropd, soildata=soil_data, sitedata=site_data)
    params["RDMCR"] = 120  # Update RDMCR in the ParameterProvider as not finding it automatically
    params["IOX"] = 1 # Update IOX in the ParameterProvider as not finding it automatically
    
    # Run simulation
    wofost = Wofost73_WLP_MLWB(params, wdp, agromanagement)
    wofost.run_till_terminate()
    output = wofost.get_output()

    # Get and print the detailed output as a DataFrame
    output_df = pd.DataFrame(output)
    print(f"Detailed output for year {year}:\n", output_df)

    # Get and print the summary output
    summary_output = wofost.get_summary_output()
    for crop_cycle in summary_output:
        print(msg.format(**crop_cycle))
    
    # Save the output to a CSV file
    output_file = os.path.join(output_dir, f"wofost_output_{year}_early.csv")
    pd.DataFrame(output).to_csv(output_file, index=False)
    print(f"Results for year {year} saved to {output_file}.")
    results.append({"year": year, "output_file": output_file})
    results_df = pd.DataFrame(results)
    summary_file = os.path.join(output_dir, "wofost_batch_run_results_early.csv")
    results_df.to_csv(summary_file, index=False)